In [11]:
import py3Dmol
from ase import io, Atoms
from ase.data import covalent_radii
import numpy as np
import networkx as nx

# Load molecule
atoms = io.read("antracene_d2h/antracene_d2h.xyz") # replace the name by your molecule XYZ file here

# Build bond graph
G = nx.Graph()
for i, pos_i in enumerate(atoms.positions):
    for j, pos_j in enumerate(atoms.positions):
        if i < j:
            dist = np.linalg.norm(pos_i - pos_j)
            cutoff = covalent_radii[atoms.numbers[i]] + covalent_radii[atoms.numbers[j]] + 0.4
            if dist < cutoff:
                G.add_edge(i, j)

# Compute bond midpoints
bond_midpoints = [(atoms.positions[i] + atoms.positions[j]) / 2 for i, j in G.edges()]

# Compute ring centroids
rings = nx.cycle_basis(G)
ring_centroids = [np.mean([atoms.positions[idx] for idx in ring], axis=0) for ring in rings]

# Prepare py3Dmol viewer
view = py3Dmol.view(width=900, height=600)

# Add molecule atoms as spheres
for atom in atoms:
    view.addSphere({'center': {'x': atom.position[0], 'y': atom.position[1], 'z': atom.position[2]},
                    'radius': 0.3,
                    'color': 'grey'})

# Add bonds as sticks
for i, j in G.edges():
    pos1 = atoms.positions[i]
    pos2 = atoms.positions[j]
    view.addCylinder({
        'start': {'x': pos1[0], 'y': pos1[1], 'z': pos1[2]},
        'end': {'x': pos2[0], 'y': pos2[1], 'z': pos2[2]},
        'radius': 0.05,
        'color': 'grey'
    })

# Add bond midpoints as red spheres + labels
for idx, mp in enumerate(bond_midpoints, start=1):
    view.addSphere({'center': {'x': mp[0], 'y': mp[1], 'z': mp[2]},
                    'radius': 0.15,
                    'color': 'red'})
    view.addLabel(str(idx), {'position': {'x': mp[0], 'y': mp[1], 'z': mp[2]+0.2}, 'backgroundColor':'black'})

# Add ring centroids as blue spheres + labels
for idx, rc in enumerate(ring_centroids, start=1):
    view.addSphere({'center': {'x': rc[0], 'y': rc[1], 'z': rc[2]},
                    'radius': 0.25,
                    'color': 'blue'})
    view.addLabel(f"R{idx}", {'position': {'x': rc[0], 'y': rc[1], 'z': rc[2]+0.3}, 'backgroundColor':'black'})

view.setStyle({'stick':{'radius':0.05}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [13]:
# Combine all points for selection
all_points = {}
for idx, mp in enumerate(bond_midpoints, start=1):
    all_points[str(idx)] = mp
for idx, rc in enumerate(ring_centroids, start=1):
    all_points[f"R{idx}"] = rc

# Ask user which points to append
print("Enter the indices/names of points to append (e.g., 9,R1,5,R2,1):")
user_input = input()
selected_keys = [x.strip() for x in user_input.split(",")]

# Get selected coordinates
selected_coords = [all_points[key] for key in selected_keys if key in all_points]

# Ask user for step size of points
print("Enter the spacing (in Å) between interpolated points (suggested: 0.02):")
step_size = float(input())

# Ask user for z-offset 
print("Enter the z-offset to apply to all Bq points (e.g., 1.0 for 1 angstrom above):")
z_offset = float(input())

# Interpolate points between consecutive selections
all_interpolated = []

for i in range(len(selected_coords) - 1):
    start = selected_coords[i]
    end = selected_coords[i + 1]
    dist = np.linalg.norm(end - start)

    # number of steps based on distance and step size
    n_steps = int(np.ceil(dist / step_size))

    # interpolate (include start + end)
    interp = np.linspace(start, end, n_steps + 1)

    # store actual interval
    actual_step = np.linalg.norm(interp[1] - interp[0])
    print(f"Actual interval for segment {i+1}: {actual_step:.4f} Å")

    if i == 0:
        all_interpolated.extend(interp)
    else:
        all_interpolated.extend(interp[1:])  # avoid duplicates

all_interpolated = np.array(all_interpolated)

# Apply z-offset
all_interpolated[:, 2] += z_offset

print(f"Total points with ~{step_size} Å spacing: {len(all_interpolated)}")

# Create ASE Atoms object for interpolated points (dummy X)
dummy_atoms = Atoms(positions=all_interpolated, symbols=['X']*len(all_interpolated))

# Combine original molecule + dummy atoms
combined = atoms + dummy_atoms

# Save combined molecule 
combined.write("antracene_d2h_bq_traj.xyz") # replace the name by your molecule
print(f"Saved {len(all_interpolated)} points appended to 'antracene_d2h_bq_traj.xyz' with z-offset {z_offset} Å")

Enter the indices/names of points to append (e.g., 9,R1,5,R2,1):


 1,R3,8,R2,17,R1,20


Enter the spacing (in Å) between interpolated points (suggested: 0.02):


 0.02


Enter the z-offset to apply to all Bq points (e.g., 1.0 for 1 angstrom above):


 1


Actual interval for segment 1: 0.0197 Å
Actual interval for segment 2: 0.0198 Å
Actual interval for segment 3: 0.0197 Å
Actual interval for segment 4: 0.0197 Å
Actual interval for segment 5: 0.0198 Å
Actual interval for segment 6: 0.0197 Å
Total points with ~0.02 Å spacing: 373
Saved 373 points appended to 'antracene_d2h_bq_traj.xyz' with z-offset 1.0 Å


In [14]:
# 1. Combine molecule + Bq points

# atoms = your original molecule
# all_interpolated = your Bq positions
bq_atoms = Atoms(positions=all_interpolated, symbols=['X']*len(all_interpolated))
combined = atoms + bq_atoms

# 2. Write Gaussian input manually
with open("antracene_d2h_bq_traj.com", "w") as f:
    f.write("%mem=4GB\n")
    f.write("%nprocshared=4\n")
    f.write("#p NMR=GIAO B972/6-311+G(d,p) Geom=connectivity\n\n")
    f.write("Molecule with Bq points\n\n")
    f.write("0 1\n")
    
    for sym, pos in zip(combined.get_chemical_symbols(), combined.positions):
        # replace X with Bq directly here
        f.write(f"{'Bq' if sym=='X' else sym:<2} {pos[0]:12.6f} {pos[1]:12.6f} {pos[2]:12.6f}\n")
    
    # Minimal connectivity to prevent auto-generation
    f.write("\n")
    for i in range(len(combined)):
        f.write(f"{i+1}\n")